## Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       

In [ ]:
import pandas as pd
import numpy as np

training = pd.read_csv("file:///Users/qiankunhuang/Downloads/project/train.csv")
location = pd.read_csv("file:///Users/qiankunhuang/Downloads/project/latlons.csv")
address = pd.read_csv("file:///Users/qiankunhuang/Downloads/project/addresses.csv")
testing = pd.read_csv("file:///Users/qiankunhuang/Downloads/project/test.csv")

In [2]:
both_col=['address','fine_amount', 'violator_name', 'zip_code',
                   'country', 'city','inspector_name',
                   'violation_street_number',
                   'violation_street_name',
                   'violation_zip_code',
                   'violation_description',
                   'mailing_address_str_number', 
                   'mailing_address_str_name',
                   'non_us_str_code','ticket_issued_date',
                   'hearing_date', 'grafitti_status',
                   'violation_code']
train_col = ['payment_amount','payment_date','payment_status','balance_due','collection_status'
,'compliance_detail','collection_status']

In [3]:
from collections import Counter
locat =location.merge(address, on = 'address',how ='inner')
train = locat.merge(training, on = 'ticket_id', how= 'inner')
train.set_index('ticket_id', inplace = True)
test = locat.merge(testing, on = 'ticket_id', how= 'inner')
test.set_index('ticket_id', inplace = True)


In [4]:
train2 = train[~train['hearing_date'].isnull()]
train2 = train2[(train2['compliance']==0)|(train2['compliance']==1)]
test = test[~test['hearing_date'].isnull()]


from datetime import datetime
def time(a,b):
    if not b or type(b) != str:
            return 73
    c=datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
    d=datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
    day= d-c
    return day.days
train2['gap'] = train2.apply(lambda row: time(row['ticket_issued_date'],row['hearing_date']),axis=1)
test['gap']= test.apply(lambda row: time(row['ticket_issued_date'],row['hearing_date']),axis=1)
train2['lat'].fillna(method = 'pad',inplace = True)
train2['lon'].fillna(method = 'pad',inplace =True)
train2['state'].fillna(method='pad',inplace=True)
test.state.fillna(method = 'pad', inplace = True)
test.lat.fillna(method = 'pad', inplace = True)
test.lon.fillna(method = 'pad', inplace = True)
test = test[test['city']=='DETROIT']
train2.drop(train_col,axis=1,inplace=True )
train2.drop(both_col,axis=1,inplace=True)
test2=test.drop(both_col,axis=1)


    

In [5]:
s = ['agency_name', 'state', 'disposition']
train3= pd.get_dummies(train2,columns=s)
test3 =  pd.get_dummies(test2,columns=s)
train_y = train3['compliance']
train_x= train3.drop('compliance',axis=1)

feature_train = train3.columns.drop('compliance')
feature_train_set = set(feature_train)
for feature in set(feature_train):
    if feature not in test3:
        feature_train_set.remove(feature)
feature_train = list(feature_train_set)
x_train = train_x[feature_train]
x_test = test3[feature_train]
    

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
train_x_scale = scaler.fit_transform(x_train)
test3_scale = scaler.fit_transform(x_test)



In [7]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes = [100,20], alpha = 6,
                       random_state = 0, solver = 'lbfgs',
                       verbose = 0)
clf.fit(train_x_scale,train_y)
test_prob = clf.predict_proba(test3_scale)
test_prob


array([[ 0.99473534,  0.00526466],
       [ 0.97966236,  0.02033764],
       [ 0.99747513,  0.00252487],
       ..., 
       [ 0.99836448,  0.00163552],
       [ 0.99717825,  0.00282175],
       [ 0.99590658,  0.00409342]])

In [8]:
train2.shape

(159653, 13)

In [9]:
from sklearn.ensemble import RandomForestClassifier 
train_y2 = train2['compliance']
train_x2= train2.drop('compliance',axis=1)
test2
rfc = RandomForestClassifier()
rfc.fit(train_x_scale, train_y)
test_prob = rfc.predict_proba(test3_scale)

In [10]:
feature_train

['agency_name_Buildings, Safety Engineering & Env Department',
 'agency_name_Detroit Police Department',
 'disposition_Responsible by Determination',
 'state_fee',
 'state_SC',
 'state_MA',
 'admin_fee',
 'state_ME',
 'state_MD',
 'state_MI',
 'clean_up_cost',
 'lon',
 'state_MO',
 'state_MN',
 'state_KS',
 'judgment_amount',
 'discount_amount',
 'state_IA',
 'agency_name_Department of Public Works',
 'disposition_Responsible by Default',
 'state_AK',
 'gap',
 'disposition_Responsible by Admission',
 'state_CA',
 'disposition_Responsible (Fine Waived) by Deter',
 'lat',
 'state_NB',
 'state_ND',
 'state_DE',
 'late_fee']

In [11]:
test_prob

array([[ 1. ,  0. ],
       [ 0.6,  0.4],
       [ 0.5,  0.5],
       ..., 
       [ 0.9,  0.1],
       [ 0.6,  0.4],
       [ 0.7,  0.3]])